In [1]:
import polars

polars.enable_string_cache()

In [2]:
from autofeat import source

dataset = source.from_kaggle(
    name="house-prices-advanced-regression-techniques",
)


In [4]:
filters = (
    dataset
    .table("train.csv")
    .column("Id")
)

target = (
    dataset
    .table("train.csv")
    .column("SalePrice")
)

In [3]:
from autofeat.transform import Cast, Combine, Encode, Require, Select

transform = (
    Require(lambda table: table.name == "train.csv")
    .then(Select(exclude={"SalePrice"}))
    .then(Cast())
    .then(Encode())
    .then(Combine())
)

In [5]:
features = (
    dataset
    .apply(transform)
    .features(filters)
    .collect()
)

KeyboardInterrupt: 

In [6]:
import sklearn.feature_selection
import xgboost

selector = sklearn.feature_selection.SelectFromModel(
    xgboost.XGBRegressor(missing=float("inf")),
    max_features=None,
)

selector.fit(
    X=features.to_numpy(),
    y=target.data.collect().to_series().to_numpy(),
)

selection = [
    features.columns[i]
    for i, selected in enumerate(selector.get_support())
    if selected
]

selection

XGBoostError: [06:51:41] /workspace/src/data/data.cc:773: Check failed: labels.Shape(0) == num_row_ (1460 vs. 0) : Size of labels must equal to number of rows.
Stack trace:
  [bt] (0) /workspaces/autofeat/.venv/lib/python3.12/site-packages/xgboost/lib/libxgboost.so(+0x22dbbc) [0x7f5cd242dbbc]
  [bt] (1) /workspaces/autofeat/.venv/lib/python3.12/site-packages/xgboost/lib/libxgboost.so(+0x4a95d6) [0x7f5cd26a95d6]
  [bt] (2) /workspaces/autofeat/.venv/lib/python3.12/site-packages/xgboost/lib/libxgboost.so(+0x5c30bf) [0x7f5cd27c30bf]
  [bt] (3) /workspaces/autofeat/.venv/lib/python3.12/site-packages/xgboost/lib/libxgboost.so(+0x5cabc9) [0x7f5cd27cabc9]
  [bt] (4) /workspaces/autofeat/.venv/lib/python3.12/site-packages/xgboost/lib/libxgboost.so(XGBoosterUpdateOneIter+0x6f) [0x7f5cd233742f]
  [bt] (5) /lib/x86_64-linux-gnu/libffi.so.8(+0x6f7a) [0x7f6541c5bf7a]
  [bt] (6) /lib/x86_64-linux-gnu/libffi.so.8(+0x640e) [0x7f6541c5b40e]
  [bt] (7) /lib/x86_64-linux-gnu/libffi.so.8(ffi_call+0xcd) [0